Source: https://github.com/dsgermain/gbfs/blob/f76251ad4c754b62defc42562887724f287b73ea/gbfs.md#file-requirements

In [392]:
# import libraries
import requests
import pandas as pd
import datetime 
from google.cloud import bigquery
from google.oauth2 import service_account
import pytz

In [393]:
# empty strings get list of urls
name_list = []
url_list = []

url = "https://mds.linkyour.city/gbfs/us_tx_austin/gbfs.json"
response = requests.get(url).json()
for i in response['data']['en']['feeds']:
    name_list.append(i['name'])
    url_list.append(i['url'])

url_df = pd.DataFrame({'name' : name_list, 'url' : url_list})

try: 
    fbs_url = url_df.loc[url_df['name'] == 'free_bike_status']['url'].values[0]
except IndexError:
    print(f"ERROR: free_bike_status file does not exist for the url: {url}")
else:
    fbs_url = fbs_url.replace("'", "")
    fbs_response = requests.get(fbs_url).json()

In [394]:
fbs_response['last_updated']
# , fbs_response['ttl']

1653024997

In [395]:
bike_dict = fbs_response['data']['bikes']
bike_df = pd.DataFrame.from_dict(bike_dict)

last_updated = datetime.datetime.fromtimestamp(int(fbs_response['last_updated'])).strftime('%Y-%m-%d %H:%M:%S')
bike_df['last_updated'] = last_updated
bike_df.head()

,bike_id,vehicle_type_id,lat,lon,is_reserved,is_disabled,pricing_plan_id,last_updated
0,c8dc417a-2d21-471f-ab26-20130a31159a,1,30.26005,-97.75451,False,False,9ff58a49-0549-48e3-b57c-d7cf2c453704,2022-05-20 00:36:37
1,f1e2d98c-0384-4151-b2b4-eff24d4ad362,1,30.29068,-97.74294,False,False,9ff58a49-0549-48e3-b57c-d7cf2c453704,2022-05-20 00:36:37
2,86d16290-e3fc-44a4-9974-50bdf5f7f279,1,30.24772,-97.73359,False,False,9ff58a49-0549-48e3-b57c-d7cf2c453704,2022-05-20 00:36:37
3,11fa32b8-f392-430f-863f-01ae94ec94fa,1,30.25688,-97.74669,False,False,9ff58a49-0549-48e3-b57c-d7cf2c453704,2022-05-20 00:36:37
4,391726be-4841-4482-a698-c82a943815c2,1,30.26365,-97.74452,False,False,9ff58a49-0549-48e3-b57c-d7cf2c453704,2022-05-20 00:36:37


In [397]:
# Reference: https://cloud.google.com/bigquery/docs/samples/bigquery-load-table-dataframe

# Construct a BigQuery client object.
path = r'C:\Users\jintp\Documents\GitHub\Data-Science-Codes\api_practice\auth\bikeshare_analytics_bq.json'
credentials = service_account.Credentials.from_service_account_file(path)
table_id = 'jintaepark-portoflio-project.bikeshare_analytics.test_upload'

client = bigquery.Client(credentials=credentials)
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")

job = client.load_table_from_dataframe(bike_df, table_id, job_config=job_config)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 959 rows and 8 columns to jintaepark-portoflio-project.bikeshare_analytics.test_upload
